In [1]:
import pdfplumber

import xlwings as xw

import pandas as pd

import numpy as np

import re


In [2]:
#提取PDF中所有页的表格（Extract a table of all pages in the PDF）

f = pdfplumber.open(r"D:\就业思路\数据分析学习\作品集\茶叶数据分析作品集\资料收集\2021年中国茶叶产销形势报告2.pdf")

tables = []
all_tables = []

for i in range(len(f.pages)):
    page_tables = f.pages[i].extract_tables()
    
    for table in page_tables:
        df = pd.DataFrame(table[1:],columns = table[0])
        all_tables.append(df)
        tables.append(table)

print(tables)

[[['省份', '2021年 2020年', '增减数', '增减％'], ['江苏', '51 45 508 0', '0 65', '1 28'], ['浙江', '307 70 307 50', '0 20', '0 07'], ['安徽', '295 73 286 32', '9 41', '3 29'], ['福建', '34122 335 40', '5 82', '1 73'], ['江西', '171 80 169 00', '2 80', '1 66'], ['山东', '408 3 39 00', '1 83', '4 69'], ['河南', '208 60 2052 0', '3 40', '1 66'], ['湖才七', '545 01 513 71', '31 30', '6 09'], ['湖南', '298 10 274 00', '24 10', '8 80'], ['广东', '123 13 104 08', '19 05', '18 30'], ['广西', '142 44 118 23', '242 1', '20 48'], ['海南', '3 35 3 32', '00 3', '09 8'], ['重庆', '84 62 782 0', '6 42', '8 21'], ['四川', '596 20 586 00', '10 20', '1 74'], ['贵州', '714 60 716 31', '-1 71', '-0 24'], ['云南', '72025 709 70', '10 55', '1 49'], ['陕西', '233 66 233 00', '06 6', '0 28']], [['省份 2021年 2020年', '增减数', '增减％'], ['江苏 10703 12000', '-1297', '-10 8'], ['浙江 195300 188100', '7200', '3 8'], ['安徽 142413 138900', '3513', '25'], ['福建 450470 418131', '32339', '77'], ['江西 78888 78076', '812', '1 0'], ['山东 27262 29600', '-2338', '-7 9'], ['河南 89190

In [3]:
print(all_tables)

[     省份    2021年 2020年    增减数    增减％
0    江苏    51 45 508 0   0 65   1 28
1    浙江  307 70 307 50   0 20   0 07
2    安徽  295 73 286 32   9 41   3 29
3    福建   34122 335 40   5 82   1 73
4    江西  171 80 169 00   2 80   1 66
5    山东    408 3 39 00   1 83   4 69
6    河南  208 60 2052 0   3 40   1 66
7   湖才七  545 01 513 71  31 30   6 09
8    湖南  298 10 274 00  24 10   8 80
9    广东  123 13 104 08  19 05  18 30
10   广西  142 44 118 23  242 1  20 48
11   海南      3 35 3 32   00 3   09 8
12   重庆    84 62 782 0   6 42   8 21
13   四川  596 20 586 00  10 20   1 74
14   贵州  714 60 716 31  -1 71  -0 24
15   云南   72025 709 70  10 55   1 49
16   陕西  233 66 233 00   06 6   0 28,         省份 2021年 2020年     增减数    增减％
0       江苏 10703 12000   -1297  -10 8
1     浙江 195300 188100    7200    3 8
2     安徽 142413 138900    3513     25
3     福建 450470 418131   32339     77
4       江西 78888 78076     812    1 0
5       山东 27262 29600   -2338   -7 9
6       河南 89190 81000    8190   10 1
7    湖才七 384000 350571   334

In [4]:
#——————第二个表格:2021年中国各主要产茶省干毛茶总产量(单位：吨)---------------

all_tables[1]

,省份 2021年 2020年,增减数,增减％
0,江苏 10703 12000,-1297,-10 8
1,浙江 195300 188100,7200,3 8
2,安徽 142413 138900,3513,25
3,福建 450470 418131,32339,77
4,江西 78888 78076,812,1 0
5,山东 27262 29600,-2338,-7 9
6,河南 89190 81000,8190,10 1
7,湖才七 384000 350571,33429,95
8,湖南 250253 240826,9427,3 9
9,广东 108443 116000,-7557,-6 5


In [6]:
#问题1：上表可见，因识别错误导致第一列是'省份 2021年 2020年'，我们需要把他们分开成三个列：'省份', '2021年', '2020年'。
#问题2：“湖北”被识别成了“湖才七”

table2 = all_tables[1]

table2[['省份', '2021年', '2020年']] = table2['省份 2021年 2020年'].str.split(' ', expand=True)
table2 = table2.drop(columns=['省份 2021年 2020年'])
table2.insert(0, '省份', table2.pop('省份'))     #将省份从第三列移动到第一列的位置
table2.insert(1, '2021年', table2.pop('2021年'))
table2.insert(2, '2020年', table2.pop('2020年'))

# 修改错误(Fix the error)
row_index = table2[table2["省份"] == '湖才七'].index[0]
table2.loc[row_index, "省份"] = '湖北' 

print("2021年中国各主要产茶省干毛茶总产量(单位：吨)")
table2

2021年中国各主要产茶省干毛茶总产量(单位：吨)


,省份,2021年,2020年,增减数,增减％
0,江苏,10703,12000,-1297,-10 8
1,浙江,195300,188100,7200,3 8
2,安徽,142413,138900,3513,25
3,福建,450470,418131,32339,77
4,江西,78888,78076,812,1 0
5,山东,27262,29600,-2338,-7 9
6,河南,89190,81000,8190,10 1
7,湖北,384000,350571,33429,95
8,湖南,250253,240826,9427,3 9
9,广东,108443,116000,-7557,-6 5


In [7]:
#将数据保存到excel

table2.to_excel('2 The total productiont of dried rough tea in China in 2021.xlsx', sheet_name='sheet 1', index=False)

In [69]:
#——————第三个表格:2021年度全国干毛茶总产值（单位：万元）---------------

table3 = all_tables[2]
table3

,省份,2021年,2020年,增减数,增减％
0,江苏,33 09,29 24,3 85,13 17
1,浙江,259 14,238 6,20 54,8 61
2,安徽,175 73,146 17,29 56,20 22
3,福建,298 12,290 42,7 70,2 65
4,江西,80 84,71 15,9 69,13 62
5,山东,74 85,30,44 85,149 50
6,河南,160 21,137 4,22 81,16 60
7,湖才七,221 91,188,33 91,18 04
8,湖南,171 57,158 27,13 30,8 40
9,广东,156 80,153 79,3 01,1 96


In [11]:
#问题一：上表所有的小数点都没有被识别出来，需要手工加一下。
#问题二：湖北”被识别成了“湖才七”

#加小数点
columns_to_fix = table3.columns[1:]  #省份不需要替换

for column in columns_to_fix:
    table3[column] = table3[column].str.replace(' ', '.').astype(float)
    
# 修改错误(Fix the error)
row_index = table3[table3["省份"] == '湖才七'].index[0]
table3.loc[row_index, "省份"] = '湖北' 

print("2022年度全国干毛茶总产值（单位：万元）")
table3

2022年度全国干毛茶总产值（单位：万元）


,省份,2021年,2020年,增减数,增减％
0,江苏,33.09,29.24,3.85,13.17
1,浙江,259.14,238.60,20.54,8.61
2,安徽,175.73,146.17,29.56,20.22
3,福建,298.12,290.42,7.70,2.65
4,江西,80.84,71.15,9.69,13.62
5,山东,74.85,30.00,44.85,149.50
6,河南,160.21,137.40,22.81,16.60
7,湖北,221.91,188.00,33.91,18.04
8,湖南,171.57,158.27,13.30,8.40
9,广东,156.80,153.79,3.01,1.96


In [12]:
#将数据保存到excel

table3.to_excel('3 Total output value of dried rough tea in 2021.xlsx', sheet_name='sheet 1', index=False)

In [98]:
#——————第四个表格:2021年中国6大茶类产量统计表----------------

table4 = all_tables[3]
table4

,茶类 2020年,2019年 增长量,增长率％
0,绿茶 184 94,184 27 0 67,0 36
1,红茶 43 45,40 43 3 02,7 47
2,黑茶 39 68,373 3 2 35,6 30
3,乌龙茶 28 72,2778 0 94,3 38
4,白茶 8 19,7 35 0 84,11 43
5,黄茶 1 33,1 45 -0 12,-8 28
6,总计 306.32,298.61 7.71,2.58


In [99]:
#问题一:两列被识别成一列
#问题二：小数点没有被识别出来

table4[["茶类","2020年"]] = table4["茶类 2020年"].str.split(' ', expand=True, n=1)
table4 = table4.drop(columns=['茶类 2020年'])
table4["2020年"] = table4["2020年"].str.replace(' ', '.').astype(float)
table4["增长率％"] = table4["增长率％"].str.replace(' ', '.').astype(float)

table4.insert(0, '茶类', table4.pop('茶类'))     #将省份从第三列移动到第一列的位置
table4.insert(1, '2020年', table4.pop('2020年'))

table4['增长率'] = table4['增长率％'] / 100

# 计算2019年数据
table4['2019年'] = table4['2020年'] / (1 + table4['增长率'])

# 计算增长量
table4['增长量'] = table4['2020年'] - table4['2019年']

table4 = table4.drop(columns=['2019年 增长量'])
table4 = table4.drop(columns=['增长率'])

table4.insert(2, '2019年', table4.pop('2019年')) 
table4.insert(3, '增长量', table4.pop('增长量')) 

print("2021年中国6大茶类产量统计表(单位：吨)")
table4


2021年中国6大茶类产量统计表(单位：吨)


,茶类,2020年,2019年,增长量,增长率％
0,绿茶,184.94,184.276604,0.663396,0.36
1,红茶,43.45,40.429887,3.020113,7.47
2,黑茶,39.68,37.328316,2.351684,6.30
3,乌龙茶,28.72,27.781002,0.938998,3.38
4,白茶,8.19,7.349906,0.840094,11.43
5,黄茶,1.33,1.450065,-0.120065,-8.28
6,总计,306.32,298.615715,7.704285,2.58


In [100]:
#将数据保存到excel

table4.to_excel('4 Chinese 6 categories tea production in 2021.xlsx', sheet_name='sheet 1', index=False)

In [102]:
#——————第五个表格:2021年中国6大茶类内销量统计表 单位：万吨----------------

table5 = all_tables[5]
table5

,茶类,2021年,2020年,增长量
0,绿茶,130 92,127 91,3 01
1,红茶,33 88,31 48,2 50
2,黑茶,34 41,31 38,2 93
3,乌龙茶,22 79,21 92,0 87


In [104]:
#问题一：小数点未被识别出来
#问题二：表格缺失了3行和一列未被识别出来

#加小数点
columns_to_fix = table5.columns[1:]

for column in columns_to_fix:
    table5[column] = table5[column].str.replace(' ', '.').astype(float)

# 将新行添加到dataframe的末尾    
new_rows = [
    ['白茶', 7.05, 6.25, 0.80],
    ['黄茶', 1.14, 1.23, -0.09],
    ['总计', 230.19, 220.17, 10.02]
]

for new_row in new_rows:
    table5.loc[table5.shape[0]] = new_row  #table5.shape是给出这个dataframe的结构，如（4,5）是4行5列，所以[0]是指行的索引。
    
# 增加新列
table5['增长率％'] = table5['增长量'] / table5['2020年'] * 100

print('2021年中国6大茶类内销量统计表 单位：万吨')
table5


,茶类,2021年,2020年,增长量,增长率％
0,绿茶,130.92,127.91,3.01,2.353217
1,红茶,33.88,31.48,2.50,7.941550
2,黑茶,34.41,31.38,2.93,9.337157
3,乌龙茶,22.79,21.92,0.87,3.968978
4,白茶,7.05,6.25,0.80,12.800000
5,黄茶,1.14,1.23,-0.09,-7.317073
6,总计,230.19,220.17,10.02,4.551029


In [105]:
#将数据保存到excel

table5.to_excel('5 Chinese 6 categories tea domestic sales of tea in 2021.xlsx', sheet_name='sheet 1', index=False)

In [107]:
#——————第六个表格:2021年中国6大茶类内销额统计表 单位：亿元----------------

table6 = all_tables[6]
table6

,茶类,2021年,2020年,增长量
0,绿茶,19934,1629 9,295 1
1,红茶,503 0,5080,22
2,黑茶,258 2,301 6,-443
3,乌龙茶,259 2,280 7,-21 5
4,白茶,91 4,89 5,1 9
5,黄茶,139,170,-31
6,总计,3120.0,2888.8,231.2


In [119]:
#问题一：未识别出小数点
#问题二：个别数字需要改动

#加小数点
columns_to_fix = table6.columns[1:]

for column in columns_to_fix:
    table6[column] = table6[column].str.replace(' ', '.').astype(float)

#修改其中某一个数值
# df.loc[row_index, 'column_name'] = new_value 
table6.loc[0, '2021年'] = 1994.3
table6.loc[0, '2020年'] = 1699.2
table6.loc[1, '2020年'] = 500.8
table6.loc[5, '2021年'] = 13.9
table6.loc[5, '2020年'] = 17.0
table6['增长量'] = table6['2021年'] - table6['2020年']
table6['增长率％'] = table6['增长量'] / table6['2020年'] * 100
print('2021年中国6大茶类内销额统计表 单位：亿元')

table6

2021年中国6大茶类内销额统计表 单位：亿元


,茶类,2021年,2020年,增长量,增长率％
0,绿茶,1994.3,1699.2,295.1,17.366996
1,红茶,503.0,500.8,2.2,0.439297
2,黑茶,258.2,301.6,-43.4,-14.389920
3,乌龙茶,259.2,280.7,-21.5,-7.659423
4,白茶,91.4,89.5,1.9,2.122905
5,黄茶,13.9,17.0,-3.1,-18.235294
6,总计,3120.0,2888.8,231.2,8.003323


In [114]:
#将数据保存到excel

table6.to_excel('6 Chinese 6 categories tea domestic sales value of tea in 2021.xlsx', sheet_name='sheet 1', index=False)

In [100]:
#——————第七个表格:2021年中国出口茶叶排名前20国家或地区   ----------------

table7 = all_tables[11]
table7

,序号 国家或地区,总量（干克）,序号,国家或地区,总额（美元）
0,1 摩洛哥,7460954,4 1,中国香港,6479168
1,2 乌兹别克斯坦,2865516,5 2,马来西亚,2566342
2,3 加纳,2283194,0 3,摩洛哥,2282593
3,4 俄罗斯联邦,1816376,9 4,越南,1283903
4,5 毛里塔尼亚,1756332,1 5,加纳,1033099
5,6 中国香港,1672917,6 6,毛里塔尼亚,7600088
6,7 塞内加尔,1634049,2 7,塞内加尔,6993257
7,8 阿尔及利亚,1306059,8 8,美国,6691187
8,9 多哥,1203877,8 9,乌兹别克斯坦,5609987
9,10 美国,1098044,5 10,多哥,5602069


In [37]:
#问题一：DataFrame中有重复的列名
#问题二：'序号'和'国家或地区'融合在了一起，需要分开
#问题三：需要替换整列数据
#问题四：还有一部分数据在另一页，需要合并

In [106]:
#DataFrame中不能有重复的列名，否则会覆盖后一列
# 第0列和第3列的列名是重复的，暂时分别命名为'序号1 国家或地区1'和'序号2 国家或地区2'
table7 = table7.rename(columns={table7.columns[0]: '序号1 国家或地区1', table7.columns[3]: '国家或地区2'})
table7[["序号1","国家或地区1"]] = table7["序号1 国家或地区1"].str.split(' ', expand = True)

table7 = table7.drop(columns=["序号1 国家或地区1"])

table7.insert(0,"序号1",table7.pop("序号1"))
table7.insert(1,"国家或地区1",table7.pop("国家或地区1"))

table7["序号2"] = list(range(1, 12))
table7 = table7.drop(columns=["序号"])
table7.insert(3,'序号2',table7.pop("序号2"))
table7

,序号1,国家或地区1,总量（干克）,序号2,国家或地区2,总额（美元）
0,1,摩洛哥,74609544,1,中国香港,647916869
1,2,乌兹别克斯坦,28655165,2,马来西亚,256634239
2,3,加纳,22831940,3,摩洛哥,228259395
3,4,俄罗斯联邦,18163769,4,越南,128390397
4,5,毛里塔尼亚,17563321,5,加纳,103309909
5,6,中国香港,16729176,6,毛里塔尼亚,76000882
6,7,塞内加尔,16340492,7,塞内加尔,69932572
7,8,阿尔及利亚,13060598,8,美国,66911878
8,9,多哥,12038778,9,乌兹别克斯坦,56099872
9,10,美国,10980445,10,多哥,56020695


In [117]:
#查看剩下的表格

table7_nextpage =pd.DataFrame(tables[12],columns = ['序号1','国家或地区1','总量（干克）','序号2','国家或地区2','总额（美元）'])
table7_nextpage

,序号1,国家或地区1,总量（干克）,序号2,国家或地区2,总额（美元）
0,12,日本,9972681,12,俄罗斯联邦,5162564
1,13,贝宁,8187896,13,阿尔及利亚,5026847
2,14,马来西亚,7242628,14,德国,4363045
3,15,喀麦隆,7172665,15,泰国,3999932
4,16,尼日尔,6011703,16,马里,2477842
5,17,泰国,6006905,17,冈比亚,2133924
6,18,冈比亚,5973665,18,尼日尔,1803535
7,19,波兰,5893928,19,利比亚,1656337
8,20,马里,5495983,20,法国,1644827


In [122]:
#合并两个表格
table7_all = pd.concat([table7,table7_nextpage])

#更改数据
table7_all["总额（美元）"] = [647916869,256634239,228259395,128390397,103309909,76000882,69932572,66911878,56099872,56020695,54376307,51625649,50268470,43630451,39999326,24778424,21339248,18035351,16563375,16448272]
table7_all

,序号1,国家或地区1,总量（干克）,序号2,国家或地区2,总额（美元）
0,1,摩洛哥,74609544,1,中国香港,647916869
1,2,乌兹别克斯坦,28655165,2,马来西亚,256634239
2,3,加纳,22831940,3,摩洛哥,228259395
3,4,俄罗斯联邦,18163769,4,越南,128390397
4,5,毛里塔尼亚,17563321,5,加纳,103309909
5,6,中国香港,16729176,6,毛里塔尼亚,76000882
6,7,塞内加尔,16340492,7,塞内加尔,69932572
7,8,阿尔及利亚,13060598,8,美国,66911878
8,9,多哥,12038778,9,乌兹别克斯坦,56099872
9,10,美国,10980445,10,多哥,56020695


In [124]:
#将数据保存到excel

table7_all.to_excel('7 top 20 countries or regions in terms of chinese tea exports.xlsx', sheet_name='sheet 1', index=False)

In [125]:
#——————第八个表格:2021年中国出口茶叶量价额统计表 ——----------------

table8 = all_tables[13]
table8

,茶类,出口量\n（吨）,出口额\n（万美元）,出口均价\n（美元／干克,出口量\n）同比增长％,出口额\n同比增长％,出口均价\n同比增长％
0,花茶,5835,5776,99,-48 2,-49 0,-00 9
1,绿茶,31226,4 14875,2 48,64 3,1396,7 08
2,乌龙茶,19145,28169,14 7,130 0,304 6,154 5
3,普泪茶,2176,5253,241,-38 6,2 -5928,-2430
4,红茶,29591,41484,14 0,27 4,204 7,172 6
5,黑茶,344,489,14 2,,,


In [ ]:
#问题一：小数点未被识别
#问题二：个别数据识别错误
#问题三：合计行缺失

In [141]:
 #因各行数据识别错误率较高，所以直接修改各行数据
    
table8.iloc[0, 1:] = [5835, 5776, 9.9, -4.82, -4.90, -0.09]
table8.iloc[1, 1:] = [312264, 148752, 4.8, 6.43, 13.96,7.08]
table8.iloc[2, 1:] = [19145, 28169, 14.7, 13.00, 30.46, 15.45]
table8.iloc[3, :] = ['普洱茶',2176, 5253, 24.1, -38.62, -52.98, -23.40]
table8.iloc[4, 1:] = [29591, 41484, 14.0, 2.74, 20.47, 17.26]
table8.iloc[5, :] = ["黑茶",344, 489, 14.2,np.nan,np.nan,np.nan]
table8.iloc[6] = pd.Series(["合计", 369355, 229923, 6.2, 5.89, 12.82, 6.55], index=table8.columns) 

#方法二 table8.loc[table8.shape[0]] = pd.Series(["合计", 369355, 229923, 6.2, 5.89, 12.82, 6.55], index=table8.columns) #table8.shape是给出这个dataframe的结构，shepe返回（4,5）是4行5列的意思，所以[0]是指行（4）的索引。
table8
                                                           

,茶类,出口量\n（吨）,出口额\n（万美元）,出口均价\n（美元／干克,出口量\n）同比增长％,出口额\n同比增长％,出口均价\n同比增长％
0,花茶,5835,5776,9.9,-4.82,-4.9,-0.09
1,绿茶,312264,148752,4.8,6.43,13.96,7.08
2,乌龙茶,19145,28169,14.7,13.0,30.46,15.45
3,普洱茶,2176,5253,24.1,-38.62,-52.98,-23.4
4,红茶,29591,41484,14.0,2.74,20.47,17.26
5,黑茶,344,489,14.2,NaN,NaN,NaN
6,合计,369355,229923,6.2,5.89,12.82,6.55


In [142]:
#将数据保存到excel

table8.to_excel('8 Chinese export tea volume and price in 2021.xlsx', sheet_name='sheet 1', index=False)

In [143]:
#——————第九个表格:2021年中国各省茶叶出口量统计表 单位：千克 ——----------------

table9 = all_tables[14]
table9

,省份,2021年出口量,2020年出口,量 增加量,增长率％
0,浙江,15080997,9 14616752,5 4642454,31 8%
1,安徽,67738272,6641744,8 1320824,1 99%
2,湖南,41557208,3549447,0 6062738,1708%
3,福建,26147684,2200492,2 4142762,188 3%
4,湖才七,23520567,1835188,1 5168686,281 6%
5,江西,14129917,1442251,9 -292602,-20 3%
6,河南,9097189,7425202,1671987,225 2%
7,四川,7652405,8226421,-574016,-69 8%
8,贵州,5937947,3276308,2661639,8124%
9,广东,5230531,6569882,-1339351,-20 39%


In [ ]:
#问题一：列名有误
#问题二：0-5行数字错位
#问题三：增长率识别错误较高，重新计算

In [149]:
#修改列名
table9 = table9.rename(columns={table9.columns[2]: '2020年出口量', table9.columns[3]: '增加量'})

#修改0-5行数据
table9.iloc[0, 1:4] = [150809979,146167525,4642454]
table9.iloc[1, 1:4] = [67738272, 66417448, 1320824]
table9.iloc[2, 1:4] = [41557208, 35494470, 6062738]
table9.iloc[3, 1:4] = [26147684, 22004922, 4142762]
table9.iloc[4, :4] = ["湖北",23520567, 18351881, 5168686]
table9.iloc[5, 1:4] = [14129917, 14422519, -292602]

#添加“增长率%”一列
table9["增加量"] = pd.to_numeric(table9["增加量"], errors='coerce')
table9["2020年出口量"] = pd.to_numeric(table9["2020年出口量"], errors='coerce')
table9["增长率％"] = table9["增加量"]/ table9["2020年出口量"] * 100
table9


,省份,2021年出口量,2020年出口量,增加量,增长率％
0,浙江,150809979,146167525,4642454,3.176119
1,安徽,67738272,66417448,1320824,1.988670
2,湖南,41557208,35494470,6062738,17.080796
3,福建,26147684,22004922,4142762,18.826524
4,湖北,23520567,18351881,5168686,28.164339
5,江西,14129917,14422519,-292602,-2.028786
6,河南,9097189,7425202,1671987,22.517731
7,四川,7652405,8226421,-574016,-6.977712
8,贵州,5937947,3276308,2661639,81.238974
9,广东,5230531,6569882,-1339351,-20.386226


In [150]:
#将数据保存到excel

table9.to_excel('9 tea exports by province in 2021 Unit: kg.xlsx', sheet_name='sheet 1', index=False)

In [4]:
#——————第十个表格:2021年中国各省茶叶出口额统计表 单位：美元 ——----------------

table10 = all_tables[15]
table10

,省份,年出口额\n2021,年出口额\n2020,增加量 增长率％
0,福建,51312885,3 41773083,3 95398020 22 84%
1,浙江,48588372,9 44722744,7 38656282 8 64%
2,安徽,28735087,6 28002712,2 7323754 262%
3,贵州,22224446,5 10357099,8 118673467 11548 %
4,湖才七,18963166,3 20lll246,0 -11480797 -57 1%
5,湖南,12420591,7 9863239,5 25573522 25 93%
6,江西,12129097,9 8471638,5 36574594 43 17%
7,云南,10846911,7 11047971,9 -2010602 -812 %
8,广东,6541701,7 7950608,8 -14089071 -1772%
9,广西,5102633,6 2872940,3 22296933 7761%


In [11]:
table10 = table10.rename(columns={ table10.columns[1]: '2021年出口额',table10.columns[2]: '2020年出口额',table10.columns[3]: '增加量'})
table10.iloc[0, 1:4] = [513128853,417730833, 95398020]
table10.iloc[1, 1:4] = [485883729, 447227447, 38656282]
table10.iloc[2, 1:4] = [287350876,280027122, 7323754]
table10.iloc[3, 1:4] = [222244465, 103570998, 118673467]
table10.iloc[4, :4] = ["湖北",189631663,201112460, -11480797]
table10.iloc[5, 1:4] = [124205917, 98632395, 25573522]
table10.iloc[6, 1:4] = [121290979, 84716385, 36574594]
table10.iloc[7, 1:4] = [108469117, 110479719, -2010602]
table10.iloc[8, 1:4] = [65417017, 79506088, -14089071]
table10.iloc[9, 1:4] = [51026336, 28729403, 22296933]
table10.iloc[10, 1:4] = [40785583, 53224585, -12439002]
table10.iloc[11, 1:4] = [29057225,15538702, 13518523]
table10.iloc[12, 1:4] = [20940671, 25613994, -4673323]
table10.iloc[13, 1:4] = [14641021, 16038473, -1397452]
table10.iloc[14, 1:4] = [10826333, 35019145, -24192812]
table10.iloc[15, 1:4] = [5731103, 19699054, -13967951]
table10.iloc[16, 1:4] = [3578548, 3641174, -62626]
table10.iloc[17, 1:4] = [3391232, 16421594, -13030362]
table10.iloc[18, 1:4] = [653960, 413288, 240672]
table10.iloc[19, 1:4] = [501416, 22128, 479288]

table10["增长率％"] = table10["增加量"]/ table10["2020年出口额"] * 100

table10

,省份,2021年出口额,2020年出口额,增加量,增长率％
0,福建,513128853,417730833,95398020,22.837199
1,浙江,485883729,447227447,38656282,8.64354
2,安徽,287350876,280027122,7323754,2.615373
3,贵州,222244465,103570998,118673467,114.581755
4,湖北,189631663,201112460,-11480797,-5.708645
5,湖南,124205917,98632395,25573522,25.928116
6,江西,121290979,84716385,36574594,43.172987
7,云南,108469117,110479719,-2010602,-1.819883
8,广东,65417017,79506088,-14089071,-17.720745
9,广西,51026336,28729403,22296933,77.610151


In [16]:
#将数据保存到excel

table10.to_excel('10 tea exports price by province in 2021 Unit dollar.xlsx', sheet_name='sheet 1', index=False)